In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import seaborn as sns
import os
from pathlib import Path

def load_and_analyze_results(file_path):
    """Charge et analyse un fichier de résultats"""
    try:
        data = np.load(file_path, allow_pickle=True).item()
        print(f"📁 Fichier: {file_path}")
        print(f"🔑 Clés disponibles: {list(data.keys()) if isinstance(data, dict) else 'Array direct'}")
        return data
    except Exception as e:
        print(f"❌ Erreur lors du chargement de {file_path}: {e}")
        return None

def create_crop_colormap():
    """Crée une colormap pour les cultures (adaptez selon vos classes)"""
    # Exemple pour PASTIS - adaptez selon votre dataset
    crop_colors = {
        0: '#000000',   # Background/No data - Noir
        1: '#FFD700',   # Wheat - Doré
        2: '#32CD32',   # Barley - Vert lime
        3: '#FF6347',   # Corn - Rouge tomate
        4: '#9370DB',   # Sunflower - Violet moyen
        5: '#FF1493',   # Rapeseed - Rose profond
        6: '#00CED1',   # Sugar beet - Turquoise
        7: '#228B22',   # Meadow - Vert forêt
        8: '#8B4513',   # Fallow - Brun selle
        9: '#FFA500',   # Soybean - Orange
        10: '#4169E1',  # Potato - Bleu royal
        11: '#DC143C',  # Bean - Rouge carmin
        12: '#ADFF2F',  # Pea - Vert jaune
        13: '#FF69B4',  # Lentil - Rose chaud
        14: '#00FF7F',  # Alfalfa - Vert printemps
        15: '#B22222',  # Clover - Rouge brique
        16: '#708090',  # Rye - Gris ardoise
        17: '#DDA0DD',  # Sorghum - Prune
        18: '#F0E68C',  # Oat - Kaki
        19: '#20B2AA'   # Mixed cereal - Vert mer clair
    }
    
    return crop_colors

def visualize_semantic_prediction(semantic_pred, title="Prédiction sémantique", 
                                save_path=None, crop_colors=None):
    """Visualise une prédiction sémantique"""
    
    if crop_colors is None:
        crop_colors = create_crop_colormap()
    
    # Créer la figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Graphique 1: Prédiction avec colormap personnalisée
    unique_classes = np.unique(semantic_pred)
    print(f"🎨 Classes détectées: {unique_classes}")
    
    # Créer une colormap personnalisée pour les classes présentes
    colors_list = [crop_colors.get(cls, '#808080') for cls in range(len(crop_colors))]
    custom_cmap = mcolors.ListedColormap(colors_list)
    
    im1 = ax1.imshow(semantic_pred, cmap=custom_cmap, vmin=0, vmax=len(crop_colors)-1)
    ax1.set_title(f"{title}\nClasses: {unique_classes}")
    ax1.axis('off')
    
    # Ajouter une colorbar
    cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8)
    cbar1.set_label('Classes de cultures')
    
    # Graphique 2: Histogramme des classes
    unique, counts = np.unique(semantic_pred, return_counts=True)
    ax2.bar(unique, counts, color=[crop_colors.get(cls, '#808080') for cls in unique])
    ax2.set_xlabel('Classe de culture')
    ax2.set_ylabel('Nombre de pixels')
    ax2.set_title('Distribution des classes')
    ax2.grid(True, alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for i, (cls, count) in enumerate(zip(unique, counts)):
        percentage = (count / semantic_pred.size) * 100
        ax2.text(cls, count, f'{count}\n({percentage:.1f}%)', 
                ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"💾 Visualisation sauvegardée: {save_path}")
    
    plt.show()
    
    return fig

def visualize_confidence_map(confidence_map, title="Carte de confiance", save_path=None):
    """Visualise une carte de confiance"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Carte de confiance
    im1 = ax1.imshow(confidence_map, cmap='viridis', vmin=0, vmax=1)
    ax1.set_title(f"{title}\nConfiance moyenne: {confidence_map.mean():.3f}")
    ax1.axis('off')
    
    cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8)
    cbar1.set_label('Niveau de confiance (0-1)')
    
    # Histogramme des confidences
    ax2.hist(confidence_map.flatten(), bins=50, alpha=0.7, color='green', edgecolor='black')
    ax2.set_xlabel('Niveau de confiance')
    ax2.set_ylabel('Nombre de pixels')
    ax2.set_title('Distribution des confidences')
    ax2.grid(True, alpha=0.3)
    
    # Statistiques
    stats_text = f"Min: {confidence_map.min():.3f}\n"
    stats_text += f"Max: {confidence_map.max():.3f}\n"
    stats_text += f"Moyenne: {confidence_map.mean():.3f}\n"
    stats_text += f"Médiane: {np.median(confidence_map):.3f}"
    
    ax2.text(0.02, 0.98, stats_text, transform=ax2.transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"💾 Carte de confiance sauvegardée: {save_path}")
    
    plt.show()
    
    return fig

def compare_fold_predictions(predictions_dict, tile_id):
    """Compare les prédictions de différents folds"""
    n_folds = len(predictions_dict)
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    crop_colors = create_crop_colormap()
    colors_list = [crop_colors.get(cls, '#808080') for cls in range(len(crop_colors))]
    custom_cmap = mcolors.ListedColormap(colors_list)
    
    for i, (fold, pred) in enumerate(predictions_dict.items()):
        if i < len(axes):
            im = axes[i].imshow(pred, cmap=custom_cmap, vmin=0, vmax=len(crop_colors)-1)
            axes[i].set_title(f'Fold {fold}\nClasses: {len(np.unique(pred))}')
            axes[i].axis('off')
    
    # Cacher les axes non utilisés
    for i in range(n_folds, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(f'Comparaison des folds - Tile {tile_id}', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return fig

def visualize_weighted_voting_results(file_path):
    """Visualise les résultats du vote pondéré"""
    data = load_and_analyze_results(file_path)
    if data is None:
        return
    
    # Extraire les données
    final_prediction = data.get('semantic_weighted_voting')
    individual_preds = data.get('individual_predictions', [])
    confidence_maps = data.get('confidence_maps', [])
    fold_confidences = data.get('fold_confidences', {})
    
    print(f"📊 Confidences par fold: {fold_confidences}")
    
    if final_prediction is not None:
        # Visualiser la prédiction finale
        tile_id = Path(file_path).stem.split('_')[0]
        visualize_semantic_prediction(
            final_prediction, 
            title=f"Vote pondéré - Tile {tile_id}",
            save_path=f"viz_{tile_id}_weighted_voting.png"
        )
        
        # Comparer avec les prédictions individuelles
        if individual_preds:
            pred_dict = {f"Fold_{i+1}": pred for i, pred in enumerate(individual_preds[:5])}
            compare_fold_predictions(pred_dict, tile_id)
        
        # Visualiser les cartes de confiance
        if confidence_maps:
            for i, conf_map in enumerate(confidence_maps[:3]):  # Premières 3 cartes
                visualize_confidence_map(
                    conf_map,
                    title=f"Confiance Fold {i+1} - Tile {tile_id}",
                    save_path=f"viz_{tile_id}_confidence_fold_{i+1}.png"
                )

def visualize_best_fold_results(file_path):
    """Visualise les résultats du meilleur fold"""
    data = load_and_analyze_results(file_path)
    if data is None:
        return
    
    if isinstance(data, dict) and 'semantic_logits' in data:
        logits = data['semantic_logits']
        confidence = data.get('confidence_score', 0)
        fold_num = data.get('fold_number', 'Unknown')
        
        # Convertir les logits en prédictions
        if len(logits.shape) == 4:  # (1, C, H, W)
            semantic_pred = np.argmax(logits[0], axis=0)
        elif len(logits.shape) == 3:  # (C, H, W)
            semantic_pred = np.argmax(logits, axis=0)
        else:
            semantic_pred = logits
        
        tile_id = Path(file_path).stem.split('_')[0]
        
        visualize_semantic_prediction(
            semantic_pred,
            title=f"Meilleur Fold {fold_num} - Tile {tile_id}\nConfiance: {confidence:.4f}",
            save_path=f"viz_{tile_id}_best_fold_{fold_num}.png"
        )

def batch_visualize_results(results_dir="preds/"):
    """Visualise tous les résultats d'un dossier"""
    results_path = Path(results_dir)
    
    if not results_path.exists():
        print(f"❌ Dossier {results_dir} introuvable")
        return
    
    # Créer dossier de visualisations
    viz_dir = Path("visualizations")
    viz_dir.mkdir(exist_ok=True)
    
    npy_files = list(results_path.glob("*.npy"))
    print(f"📁 {len(npy_files)} fichiers .npy trouvés")
    
    for file_path in npy_files:
        print(f"\n🎨 Traitement de {file_path.name}")
        
        try:
            if "weighted_voting" in file_path.name:
                visualize_weighted_voting_results(str(file_path))
            elif "best_fold" in file_path.name:
                visualize_best_fold_results(str(file_path))
            else:
                # Fichier générique
                data = load_and_analyze_results(str(file_path))
                if isinstance(data, np.ndarray) and len(data.shape) == 2:
                    tile_id = file_path.stem
                    visualize_semantic_prediction(
                        data,
                        title=f"Prédiction - {tile_id}",
                        save_path=f"visualizations/viz_{tile_id}.png"
                    )
        
        except Exception as e:
            print(f"❌ Erreur avec {file_path.name}: {e}")

# Fonction principale
if __name__ == "__main__":
    # Exemples d'utilisation
    
    # 1. Visualiser tous les résultats
    #batch_visualize_results("preds/")
    
    # 2. Visualiser un fichier spécifique
     #visualize_weighted_voting_results("preds/0_weighted_voting.npy")
     visualize_best_fold_results("preds/0_best_fold_2_UTAE_semantic.npy")

📁 Fichier: preds/0_best_fold_2_UTAE_semantic.npy
🔑 Clés disponibles: ['semantic_pred', 'confidence_map', 'raw_output']
